In [1]:
import pandas as pd
from sklearn.ensemble import HistGradientBoostingClassifier as HGBC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import random

cols = ["id", "Location", "Name", "Address", "District", "RoadInfo", "Poi", "RoomInfo", "label"]

dfPos = pd.read_csv("./train_data/positive.csv")
dfNeg = pd.read_csv("./train_data/negative.csv")

df = pd.concat([dfPos, dfNeg], axis=0)
X = df[cols[:-1]].values
y = df["label"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)
testId = [x[0] for x in X_test]
X_train = [x[1:] for x in X_train]
X_test = [x[1:] for x in X_test]

In [25]:
clf = XGBClassifier()
clf = clf.fit(X_train, y_train)
result = clf.score(X_test, y_test)
pred = clf.predict(X_test)
print(result)
feature_importance = clf.feature_importances_
print("特征重要性：", feature_importance)
# print(clf.get_booster().get_score(importance_type= "gain"))
# predictions_proba = clf.predict_proba(X_test)
# for lst in predictions_proba:
#     print("%.3f, %.3f"%(lst[0], lst[1]))

0.9666666666666667
特征重要性： [0.16519575 0.67253864 0.07393469 0.03268984 0.05564112 0.
 0.        ]


In [4]:
# 输出测试数据预测结果

import json
dataName1 = "Ctrip"
dataName2 = "Mafengwo"
with open("./train_data/%s_addr.json"%dataName1, "r", encoding="utf-8") as f:
    data1 = json.load(f)
with open("./train_data/%s_addr.json"%dataName2, "r", encoding="utf-8") as f:
    data2 = json.load(f)

result = []
cnt = -1
for i in testId:
    id = i.split("-")
    id1, id2 = int(id[0][1:]), int(id[1][1:])
    d1, d2 = data1[id1], data2[id2]
    cnt += 1
    lst1, lst2 = [cnt, "C"+str(id1)], [cnt, "M"+str(id2)]
    for attr in cols[1:-1]:
        lst1.append(d1[attr])
        lst2.append(d2[attr])
    lst1[2] = str(lst1[2][0]) + ", " + str(lst1[2][1])
    lst2[2] = str(lst2[2][0]) + ", " + str(lst2[2][1])
    lst1.append(y_test[cnt])
    if pred[cnt] == 1: lst2.append(True)
    else: lst2.append(False)
    result.append(lst1)
    result.append(lst2)
    result.append(["" for _ in range(10)])
dfResult = pd.DataFrame(result, columns=["cluster"]+cols[:-1]+["real & pred"])
dfResult.to_csv("test_result.csv", encoding="utf-8")

In [5]:
import json

dataName1 = "Mafengwo"
dataName2 = "Tripadvisor"
with open("./user_data/%s_addr.json"%dataName1, "r", encoding="utf-8") as f:
    data1 = json.load(f)
with open("./user_data/%s_addr.json"%dataName2, "r", encoding="utf-8") as f:
    data2 = json.load(f)

def combineEnt(d1, d2, label, prob):
    rnt = {"id": "%s-%s"%(d1["id"], d2["id"])}
    rnt["label"] = label
    rnt["Probability"] = prob
    for attr in list(d1.keys())[1:]:
        rnt[attr] = [d1[attr], d2[attr]]
    return rnt

dfTest = pd.read_csv("./user_data/data.csv")
test = dfTest[cols[1:-1]].values
pred = clf.predict(test)
predProb = clf.predict_proba(test).tolist()
idList = list(dfTest["id"])

result = []
for i in range(len(idList)):
    idx = idList[i].split("-")
    idx1, idx2 = int(idx[0][1:]), int(idx[1][1:])
    d1 = data1[idx1]; d2 = data2[idx2]
    if pred[i] == True:
        result = [combineEnt(d1, d2, True, predProb[i])] + result
    else:
        result.append(combineEnt(d1, d2, False, predProb[i]))

result = sorted(result, key=lambda x:x["Probability"][1], reverse=True)

with open("./result.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)


In [11]:
print(pred.tolist().count(1))

56
